In [30]:
%%writefile app.py

from pyngrok import ngrok
import streamlit as st
import pandas as pd
from io import BytesIO
import re
import time
from openpyxl import load_workbook
import os

# ---------------- STREAMLIT APP ---------------- #

st.title("Excel Form Processor")

# Step 1: Upload Excel file
uploaded_file = st.file_uploader("Upload an Excel file", type=["xlsx"])

if uploaded_file:

    original_file_name = os.path.basename(uploaded_file.name)

    # Use session_state to avoid reprocessing on download click
    if 'processed_output' not in st.session_state:

        xls = pd.ExcelFile(uploaded_file)

        # Ensure required sheets exist
        required_sheets = ["formFields", "transitions", "workflowSlas"]
        for sheet in required_sheets:
            if sheet not in xls.sheet_names:
                st.error(f"Missing required sheet: '{sheet}'")
                st.stop()

        # ---------------- Load Sheets ---------------- #
        df = pd.read_excel(uploaded_file, sheet_name="formFields")
        transitions_df = pd.read_excel(uploaded_file, sheet_name="transitions")
        workflow_slas_df = pd.read_excel(uploaded_file, sheet_name="workflowSlas")

        # Required column names
        field_name_col = "name"
        field_type_col = "input_type"
        field_order_col = "order"
        form_col = "formName"

        fixed_category_cols = [
            "is_editable", "is_hidden", "is_required", "is_nullable",
            "is_multiselect", "is_richtext", "editor_height",
            "is_encrypted", "is_conditional"
        ]
        selected_category_cols = [c for c in fixed_category_cols if c in df.columns]

        # Load mapping sheet
        field_mapping_df = pd.DataFrame(columns=["formName", "fieldName"])
        if "fieldMapping" in xls.sheet_names:
            field_mapping_df = pd.read_excel(uploaded_file, sheet_name="fieldMapping")

        # Process forms automatically
        all_forms = sorted(df[form_col].dropna().unique())
        num_forms = len(all_forms)

        output = BytesIO()
        progress_bar = st.progress(0)

        uploaded_file.seek(0)
        wb = load_workbook(uploaded_file)

        # ---------- Live sheet creation UI ----------
        sheet_list_placeholder = st.empty()
        created_sheets = []

        st.info(f"Created {num_forms + 1} sheets")  # +1 for Transition_mapping

        # ============================================================
        #                     PROCESS FORMS
        # ============================================================

        for i, full_form_name in enumerate(all_forms):

            parts = full_form_name.split("_")
            if len(parts) > 3:
                middle_parts = parts[2:-1]
                short_name = "_".join(middle_parts)
            else:
                short_name = full_form_name

            safe_short_name = re.sub(r'[\\/*?:\[\]]', '_', short_name)[:31]

            form_df = df[df[form_col] == full_form_name].sort_values(by=field_order_col)
            final_df = form_df[[field_name_col, field_type_col, field_order_col]].copy()

            # Add Repeater column
            repeater_list = []
            filtered_mapping = field_mapping_df[field_mapping_df['formName'] == full_form_name]
            mapped_fields = set(filtered_mapping['fieldName'].dropna())
            for field in final_df[field_name_col]:
                repeater_list.append("Yes" if field in mapped_fields else "")
            final_df.insert(len(final_df.columns), "Repeater", repeater_list)

            # Add Categories column
            if selected_category_cols:
                categories_list = []
                for idx, row in form_df.iterrows():
                    selected = [col for col in selected_category_cols if row.get(col) == 1]
                    selected.sort()
                    categories_list.append(",".join(selected) if selected else "")
                final_df['Categories'] = categories_list

            if safe_short_name in wb.sheetnames:
                wb.remove(wb[safe_short_name])
            ws = wb.create_sheet(title=safe_short_name)

            ws.cell(row=1, column=1, value="Form")
            ws.cell(row=1, column=2, value=full_form_name)

            for c_idx, header in enumerate(final_df.columns, start=1):
                ws.cell(row=2, column=c_idx, value=header)
            for r_idx, row in enumerate(final_df.values, start=3):
                for c_idx, value in enumerate(row, start=1):
                    ws.cell(row=r_idx, column=c_idx, value=value)

            # Update progress
            progress = int((i + 1) / num_forms * 100)
            progress_bar.progress(progress)

            # Update live sheet list
            created_sheets.append(safe_short_name)
            with sheet_list_placeholder.container():
                st.markdown("**Sheets created so far:**")
                st.markdown(
                    "<div style='max-height:250px; overflow-y:auto;'>"
                    + "<br>".join([f"{idx+1}. {name}" for idx, name in enumerate(created_sheets)])
                    + "</div>",
                    unsafe_allow_html=True
                )

            time.sleep(0.05)

        # After all forms done, remove the "Sheets created so far" info
        sheet_list_placeholder.empty()

        # ============================================================
        #              TRANSITION_MAPPING SHEET
        # ============================================================

        transition_cols = [
            "name",
            "workflowFromStateName",
            "workflowToStateName",
            "workflowFormName",
            "fromToStateName"
        ]
        transition_df = transitions_df[transition_cols].copy()

        # Case-insensitive alphabetical sorting
        transition_df.sort_values(
            by=["workflowFromStateName", "workflowToStateName"],
            key=lambda col: col.str.lower(),
            inplace=True
        )

        merged_df = pd.merge(
            transition_df,
            workflow_slas_df[
                ["workflowFromStateName", "workflowToStateName", "sla_time", "sla_time_type"]
            ],
            on=["workflowFromStateName", "workflowToStateName"],
            how="left"
        )

        final_cols = [
            "name",
            "workflowFromStateName",
            "workflowToStateName",
            "workflowFormName",
            "sla_time",
            "sla_time_type",
            "fromToStateName"
        ]
        merged_df = merged_df[final_cols]

        if "Transition_mapping" in wb.sheetnames:
            wb.remove(wb["Transition_mapping"])
        ws_map = wb.create_sheet("Transition_mapping")

        for c_idx, header in enumerate(final_cols, start=1):
            ws_map.cell(row=1, column=c_idx, value=header)
        for r_idx, row in enumerate(merged_df.values, start=2):
            for c_idx, value in enumerate(row, start=1):
                ws_map.cell(row=r_idx, column=c_idx, value=value)

        # Add Transition_mapping to final list
        created_sheets.append("Transition_mapping")

        # ============================================================
        # Save final workbook in session_state
        # ============================================================
        wb.save(output)
        st.session_state.processed_output = output.getvalue()
        st.session_state.created_sheets = created_sheets

    # Use processed_output from session_state for download
    st.success(f"Modified Excel created successfully!")
    st.download_button(
        label="Download Final Excel",
        data=st.session_state.processed_output,
        file_name=f"Processed_{original_file_name}",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )

    # Show final sheets list after processing and download
    with st.container():
        st.markdown("**Sheets created:**")
        st.markdown(
            "<div style='max-height:250px; overflow-y:auto;'>"
            + "<br>".join([f"{idx+1}. {name}" for idx, name in enumerate(st.session_state.created_sheets)])
            + "</div>",
            unsafe_allow_html=True
        )


Overwriting app.py


In [31]:
# Install packages (if not installed already)
!pip install streamlit pyngrok openpyxl --quiet

from pyngrok import ngrok
import time

# Kill previous tunnels
ngrok.kill()

# Set your ngrok token
ngrok.set_auth_token("35gXlCpbnwCAIJ9I2xiA1GXDu2u_3UoQf6QeGYhFC9MxYapgo")

# Start Streamlit in the background
get_ipython().system_raw("streamlit run app.py &")

# Wait a few seconds for Streamlit to start
time.sleep(5)

# Open ngrok tunnel
public_url = ngrok.connect(8501)
print("Streamlit app running at:", public_url)


Streamlit app running at: NgrokTunnel: "https://armless-twirly-alanna.ngrok-free.dev" -> "http://localhost:8501"
